In [2]:
import tushare as ts
import os
import datetime
import pymongo
import pandas
import copy

def ConnectDB():
    client = pymongo.MongoClient("127.0.0.1", 27017)
    return client

def Today():
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    return today

def SaveResult(client, code, date, strategy, data):
    db = client.Result
    collection = db.Recommend
    collection.replace_one({
        "code": code,
        "date": date,
        "strategy": strategy
    }, data, upsert=True)
    

In [3]:
def Strategy_1():
    client = ConnectDB()
    
    startDate = datetime.datetime.now() + datetime.timedelta(days=-60)
    stocks = client.Common.Stock.find({"foundTime": {"$lt": startDate}})
    threshold = 9.9
    today = Today()
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({}, limit=4, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 4:
            continue
        changes = []
        for i in range(3):
            if res[i+1]["close"] == 0:
                d = 0
            else:
                d = (res[i]["close"] / res[i+1]["close"] - 1) * 100
            changes.append(d)
        if changes[0] < threshold or (changes[1] <= threshold and changes[2] <= threshold):
            continue
        days = (today - res[3]["date"]).days
        if days > minDays:
            continue
        if days < minDays:
            minDays = days
            found = []
        found.append({
            "code": code,
            "name": item["name"],
            "changes": changes
        })
        data = {
            "code": code,
            "name": item["name"],
            "date": today,
            "strategy": 1,
            "changes": changes
        }
        SaveResult(client, code, today, 1, data)

    if len(found) > 0:
        print pandas.DataFrame(found, columns=["code", "name", "changes"])
    else:
        print("No Result")
        
    client.close()

In [20]:
def Strategy_3():
    client = ConnectDB()
    
    stocks = client.Common.Stock.find({})
    today = Today()
    found = []
    minDays = 1000

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = list(collection.find({"close": {"$ne": 0}}, limit=10, sort=[("date", pymongo.DESCENDING)]))
        if len(res) < 10:
            continue
        days = (today - res[-1]["date"]).days
        if days > minDays:
            continue
        if res[0]["close"] < res[1]["close"]:
            continue
        if days < minDays:
            minDays = days
            found = []
        v10 = 0
        volumeDays = 2
        vv = 0
        p5 = 0
        for i in range(10):
            v10 = v10 + res[i]["volume"]
            if i < volumeDays:
                vv = vv + res[i]["volume"]
            if i < 5:
                p5 = p5 + res[i]["close"]
        v10 = int(v10 / 10)
        vv = int(vv / volumeDays)
        p5 = p5 / 5
        if v10 == 0:
            continue
        close = res[0]["close"]
        if vv >= v10 * 2 and close >= p5:
            item = {
                "code": code,
                "name": item["name"],
                "volume_" + str(volumeDays): vv,
                "volume_10": v10,
                "price_today": close,
                "price_5": p5,
                "factor": (1.0 * vv / v10)
            }
            found.append(item)

        found = sorted(found, key=lambda item: item["factor"], reverse=True)

    found = found[:10]
    for item in found:
        data = copy.copy(item)
        data["date"] = today
        data["strategy"] = 3
        SaveResult(client, item["code"], today, 3, data)
        
    if len(found) > 0:
        print pandas.DataFrame(found[:10])
    else:
        print("No Result")
        
    client.close()

     code    factor  name  price_5  price_today  volume_10  volume_2
0  300318  3.166681  博晖创新    5.612         6.20      71034    224942
1  600892  2.958195  大晟文化    9.870        10.18      33848    100129
2  601918  2.862827  新集能源    3.154         3.39     157597    451173
3  300519  2.652851  新光药业   20.586        23.19      23693     62854
4  603589  2.644649   口子窖   46.570        48.92      34487     91206
5  600211  2.566719  西藏药业   41.502        42.56      16622     42664
6  603088  2.472071  宁波精达   24.894        27.64      21626     53461
7  300200  2.440292  高盟新材    9.374         9.88      35590     86850
8  600329  2.375056  中新药业   15.240        16.63      44820    106450
9  600976  2.331505  健民集团   24.132        25.44       7656     17850


In [5]:
def Strategy_4():
    client = ConnectDB()
    
    today = Today()
    stocks = client.Common.Stock.find({
        "$and": [
            {"value": {"$exists": True}},
            {"value": {"$ne": 0}}
        ]})

    found = []

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = collection.find_one({"date": today})
        if not res:
            continue
        if "last_close" not in res:
            continue
        if res["close"] <= res["last_close"]:
            continue
        value = res["tradeValue"] * 10000
        if "amount" in res:
            amount = res["amount"]
        else:
            amount = res["volume"] * (res["open"] + res["close"]) / 2 * 100
        item = {
            "code": code,
            "name": item["name"],
            "value": value / 1e8,
            "amount": amount / 1e8,
            "factor": amount / value
        }
        found.append(item)
        
    found = sorted(found, key=lambda item: item["factor"], reverse=True)[:10]
    for item in found:
        data = copy.copy(item)
        data["date"] = today
        data["strategy"] = 4
        SaveResult(client, item["code"], today, 4, data)
        
    print(pandas.DataFrame(found[:10], columns=["code", "name", "factor", "amount", "value"]))

    client.close()

In [6]:
def Strategy_2():
    client = ConnectDB()
    
    today = Today()
    conceptDict = {}
    Threshold = 9.9
    
    for item in client.Common.Stock.find({}):
        code = item["code"]
        collection = client.Trade[code]
        res = collection.find_one({"date": today})
        if not res:
            continue
        if "last_close" not in res:
            continue
        change = (res["close"] / res["last_close"] - 1) * 100
        if change < Threshold:
            continue
    
        for concept in item["concept"]:
            if concept == u"次新股":
                continue
            if concept not in conceptDict:
                conceptDict[concept] = []
            data = {
                "code": item["code"],
                "name": item["name"],
                "change": change
            }
            conceptDict[concept].append(data)
    
    concepts = []
    for k, v in conceptDict.iteritems():
        if len(v) < 3:
            continue
        data = {
            "concept": k,
            "stocks": v,
            "count": len(v),
            "codes": []
        }
        for s in v:
            data["codes"].append(s["code"])
        concepts.append(data)
    
    found = sorted(concepts, key=lambda item: item["count"], reverse=True)
    
    print(pandas.DataFrame(found, columns=["concept", "count", "codes"]))
    
    stockDict = {}
    for k in found:
        for s in k["stocks"]:
            code = s["code"]
            if code not in stockDict:
                stockDict[code] = {
                    "code": code,
                    "name": s["name"],
                    "change": s["change"],
                    "concepts": [],
                    "date": today,
                    "strategy": 2,
                }
            stockDict[code]["concepts"].append(k["concept"])
    for code, v in stockDict.iteritems():
        SaveResult(client, code, today, 2, v)
        
    client.close()

In [7]:
def FilterSameStrategy():
    client = ConnectDB()
    today = Today()
    
    stocks = {}
    for s in client.Result.Recommend.find({"date": today}):
        code = s["code"]
        if code not in stocks:
            stocks[code] = {
                "code": code,
                "name": s["name"],
                "strategies": []
            }
        stocks[code]["strategies"].append(s["strategy"])
        
    result = []
    for k, v in stocks.iteritems():
        if len(v["strategies"]) > 1:
            result.append({
                "code": k,
                "name": v["name"],
                "strategies": v["strategies"]
            })
    print pandas.DataFrame(result)

In [22]:
print("Strategy 1")
Strategy_1()
print("\nStrategy 2")
Strategy_2()
print("\nStrategy 3")
Strategy_3()
print("\nStrategy 4")
Strategy_4()
print("\nSummary")
FilterSameStrategy()

Strategy 1
     code  name                                        changes
0  002458  益生股份           [10.0, 2.72108843537, 10.0299401198]
1  002795  永和智控    [9.979770735, 10.0148367953, -4.6676096181]
2  300519  新光药业  [10.009487666, 10.0208768267, -4.05608412619]
3  300526  中潜股份  [9.9886492622, 9.98751560549, -6.20608899297]

Strategy 2
Empty DataFrame
Columns: [concept, count, codes]
Index: []

Strategy 3
     code    factor  name  price_5  price_today  volume_10  volume_2
0  300318  3.166681  博晖创新    5.612         6.20      71034    224942
1  600892  2.958195  大晟文化    9.870        10.18      33848    100129
2  601918  2.862827  新集能源    3.154         3.39     157597    451173
3  300519  2.652851  新光药业   20.586        23.19      23693     62854
4  603589  2.644649   口子窖   46.570        48.92      34487     91206
5  600211  2.566719  西藏药业   41.502        42.56      16622     42664
6  603088  2.472071  宁波精达   24.894        27.64      21626     53461
7  300200  2.440292  高盟新材    9.374     

In [23]:
client = ConnectDB()
today = Today()
minDate = today + datetime.timedelta(days=-10)
collection = client.Result.Recommend
cursor = collection.find({"date": {"$gt": minDate}}) \
    .sort("date", pymongo.DESCENDING)
    
stocks = {}
daysDiff = -1
daysCount = 0
daysNeed = 4
startDate = None
for s in cursor:
    diff = (today - s["date"]).days
    if diff > daysDiff:
        daysDiff = diff
        daysCount = daysCount + 1
    if daysCount > daysNeed:
        break
    if daysCount < daysNeed:
        continue
    
    startDate = s["date"]
    code = s["code"]
    if code not in stocks:
        stocks[code] = {
            "code": code,
            "name": s["name"],
            "strategies": [s["strategy"]]
        }
    else:
        stocks[code]["strategies"].append(s["strategy"])
        
found = []
for code, stock in stocks.iteritems():
    res = []
    for s in client.Trade[code].find({"date": {"$gt": startDate}}):
        res.append(s)
    if len(res) < daysNeed - 1:
        continue
    buyPrice = res[0]["open"]
    sellPrice1 = res[1]["high"]
    sellPrice2 = res[2]["high"]
    if buyPrice == 0:
        continue
    change1 = (sellPrice1 / buyPrice - 1) * 100
    change2 = (sellPrice2 / buyPrice - 1) * 100
    found.append({
        "code": code,
        "name": stock["name"],
        "strategies": stock["strategies"],
        "change1": change1,
        "change2": change2,
        "buy": buyPrice,
        "sell1": sellPrice1,
        "sell2": sellPrice2
    })
    
found = sorted(found, key=lambda item: item["change2"], reverse=True)
print pandas.DataFrame(found, columns=["code", "name", "change1", "change2", "buy", "sell1", "sell2", "strategies"])

      code   name     change1     change2    buy  sell1  sell2 strategies
0   002877   智能自控    8.433735   27.053669  18.26  19.80  23.20        [1]
1   300634   彩讯股份    9.961686   26.436782  52.20  57.40  66.00        [4]
2   002931   锋龙股份    8.327691   24.125480  44.31  48.00  55.00     [3, 4]
3   002052   同洲电子   10.251799   23.381295   5.56   6.13   6.86     [1, 2]
4   300555   路通视信   11.217391   19.217391  11.50  12.79  13.71     [1, 2]
5   600288   大恒科技   11.256831   19.125683   9.15  10.18  10.90        [2]
6   300588   熙菱信息    8.501259   19.017632  31.76  34.46  37.80        [1]
7   300739   明阳电路    8.571429   18.985507  48.30  52.44  57.47        [4]
8   300293   蓝英装备   12.258065   18.870968  12.40  13.92  14.74     [1, 2]
9   600866   星湖科技    8.716707   18.401937   4.13   4.49   4.89        [2]
10  000836   鑫茂科技    4.523810   18.333333   4.20   4.39   4.97        [2]
11  002181  粤 传 媒    8.673469   18.197279   5.88   6.39   6.95        [1]
12  002134  *ST普林   25.658648   16.953